In [1]:
import pandas as pd
import os
import shutil
import numpy as np
import torchvision
from torchvision.transforms import ToTensor
import os
from PIL import Image
from torchvision import transforms
import torch
from tensorflow.keras.preprocessing.image import ImageDataGenerator


ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (D:\Analytics\anaconda\lib\site-packages\tensorflow\python\eager\def_function.py)

## Pre-Work

In [6]:
df_labels = pd.read_csv('dataset/train.csv')
validation_split = 0.8
train_split = 0.75
idxs = np.random.permutation(range(len(df_labels)))

train_idxs = idxs[:int(train_split * len(idxs))]
val_idxs = idxs[int(train_split * len(idxs)):int(validation_split * len(idxs))]
test_idxs = idxs[int(validation_split * len(idxs)):]

train_labels = df_labels.iloc[train_idxs]
val_labels = df_labels.iloc[val_idxs]
test_labels = df_labels.iloc[test_idxs]

print(len(train_labels), len(test_labels), len(val_labels))

6170 1646 411


In [8]:


# Define the source directory containing all files
source_dir = "dataset/train"

# Define the destination directories for train, test, and validation sets
train_dir = "train_images"
test_dir = "test_images"
val_dir = "val_images"

# Define the indices of the files in each set
train_indices = train_labels.image_ID 
test_indices = test_labels.image_ID 
val_indices = val_labels.image_ID 

# Create the destination directories if they don't exist
for dir_path in [train_dir, test_dir, val_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Loop through the files in the source directory
for i, filename in enumerate(os.listdir(source_dir)):
    # Get the full path to the source file
    src_path = os.path.join(source_dir, filename)

    # Determine which destination directory to use based on the index
    if i in train_indices:
        dest_dir = train_dir
    elif i in test_indices:
        dest_dir = test_dir
    elif i in val_indices:
        dest_dir = val_dir
    else:
        continue

    # Get the full path to the destination file
    dest_path = os.path.join(dest_dir, filename)

    # Copy the file to the destination directory
    shutil.copy(src_path, dest_path)

In [16]:


class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, target_transform=None, loader=default_loader):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img_path = os.path.join(self.root_dir, str(img_id) + '.jpg')
        image = self.loader(img_path)
        label = self.annotations.iloc[index, 1]
        
        if self.transform:
            image = self.transform(image)
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label

# Example usage
csv_file = 'dataset/train.csv'
root_dir = 'train_images'
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])
dataset = CustomDataset(csv_file=csv_file,
                        root_dir=root_dir,
                        transform=transform)
train_ds = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [18]:
csv_file = 'dataset/train.csv'
root_dir = 'test_images'
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])
dataset = CustomDataset(csv_file=csv_file,
                        root_dir=root_dir,
                        transform=transform)
test_ds = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [19]:
csv_file = 'dataset/train.csv'
root_dir = 'val_images'
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])
dataset = CustomDataset(csv_file=csv_file,
                        root_dir=root_dir,
                        transform=transform)
val_ds = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

## Modeling

In [23]:
image_to_label_map=dict(zip(train_labels["image_ID"].values,train_labels["label"].values))

In [25]:
unique_classes=train_labels["label"].unique()
print(unique_classes)
print(len(unique_classes))

['Cricket' 'Wrestling' 'Soccer' 'Badminton' 'Swimming' 'Tennis' 'Karate']
7


In [26]:
from os import makedirs
# Create Directories
subdirs=["train_images/","val_images/","test_images/"]

for subdir in subdirs:
    # create Label directories
    labeldirs=unique_classes
    for label in labeldirs:
        new_dir=subdir+label+"/"
        makedirs(new_dir,exist_ok=True)

In [34]:
# Create DataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_datagen=ImageDataGenerator(rescale=1./255)

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (D:\Analytics\anaconda\lib\site-packages\tensorflow\python\eager\def_function.py)

In [ ]:
train_directory="./train"
validation_directory="./validation"

# Dimension of our Images
img_height,img_width=224,224
batch_size=64
train_generator=train_datagen.flow_from_directory(
    train_directory,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode="categorical"
)
validation_generator=test_datagen.flow_from_directory(
    validation_directory,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode="categorical"
)